Importing Libraries

In [2]:
import cv2
import time 

import tensorflow as tf
import os
import shutil
import numpy as np
import random
from tqdm import tqdm
from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
from pathlib import Path
import re
import cv2
from keras import backend as K
tf.__version__
import glob

#check if tensorflow gpu is being used
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("tensorflow version:", tf.__version__)

Num GPUs Available:  1
tensorflow version: 2.10.0


In [11]:
#Realtime Segmentation with Dice Score and Bounding Box Detection

#Reference for bbox(Github): https://github.com/murtazahassan/OpenCV-Python-Tutorials-and-Projects/blob/master/Intermediate/RealTime_Shape_Detection_Contours.py 
#Reference for bbox (Video): https://www.youtube.com/watch?v=Fchzk1lDt7Q 

#Reference for realtime segmentation: https://anirudhtopiwala.com/my-robots/skin-segmentation-using-deep-learning/
#Github Code for realtime segmentation: https://github.com/anirudhtopiwala/abd-skin-segmentation/blob/master/Real%20Time%20Skin%20Segmentation/UNET-live.py 
#playing video on openCV: https://www.geeksforgeeks.org/python-play-a-video-using-opencv/ 
#adding FPS for openCV: https://www.geeksforgeeks.org/python-displaying-real-time-fps-at-which-webcam-video-file-is-processed-using-opencv/ 

#videos to choose from
NeedleViz_path1 = 'Data/edited data/102622_Water.mp4'
NeedleViz_path2 = 'Data/edited data/102822_Water.mp4'

#control playback speed
frame_rate = 25

# vc = cv2.VideoCapture(0) #opens camera
vc = cv2.VideoCapture(NeedleViz_path1)

prev_frame_time = 0
new_frame_time = 0
average_fps_list = []

frameWidth = 440
frameHeight = 440
vc.set(3, frameWidth)
vc.set(4, frameHeight)

def empty(a):
  pass

previous_frame = None

if (vc.isOpened()== False): 
  print("Error opening video  file")

while(vc.isOpened()):
    rval, frame = vc.read()
    
    if rval == True:
      #Insert Code Here
      
      # 1. Load image; convert to RGB
      img_brg = frame
      img_rgb = cv2.cvtColor(src=img_brg, code=cv2.COLOR_BGR2RGB)

      # 2. Prepare image; grayscale and blur
      prepared_frame = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
      prepared_frame = cv2.GaussianBlur(src=prepared_frame, ksize=(5, 5), sigmaX=0)

      # 2. Calculate the difference
      if (previous_frame is None):
        # First frame; there is no previous one yet
        previous_frame = prepared_frame
        continue

      # 3. Set previous frame and continue if there is None
      if (previous_frame is None):
        # First frame; there is no previous one yet
        previous_frame = prepared_frame
        continue

      # calculate difference and update previous frame
      diff_frame = cv2.absdiff(src1=previous_frame, src2=prepared_frame)
      previous_frame = prepared_frame

      # 4. Dilute the image a bit to make differences more seeable; more suitable for contour detection
      kernel = np.ones((5, 5))
      diff_frame = cv2.dilate(diff_frame, kernel, 1)

      # 5. Only take different areas that are different enough (>20 / 255)
      thresh_frame = cv2.threshold(src=diff_frame, thresh=70, maxval=255, type=cv2.THRESH_BINARY)[1]

      # 6. Find and optionally draw contours
      contours, _ = cv2.findContours(image=thresh_frame, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)
      # Comment below to stop drawing contours
      cv2.drawContours(image=img_rgb, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
      # Uncomment 6 lines below to stop drawing rectangles
      for contour in contours:
        if cv2.contourArea(contour) < 70:
          # too small: skip!
            continue
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(img=img_rgb, pt1=(x, y), pt2=(x + w, y + h), color=(0, 255, 0), thickness=2)

      cv2.imshow('Motion detector', img_rgb)

      # Press Q on keyboard to  exit
      if cv2.waitKey(frame_rate) & 0xFF == ord('q'): #original waitkey is 25
          break
    
    #Break out of loop if video is done
    else:
        break  

vc.release() #Release the video capture object

# Close window
cv2.destroyWindow("Motion detector")


In [15]:
#Background Subtraction
#reference: https://www.geeksforgeeks.org/python-background-subtraction-using-opencv/?ref=lbp, 


#videos to choose from
NeedleViz_path1 = 'Data/edited data/102622_Water.mp4'
NeedleViz_path2 = 'Data/edited data/102822_Water.mp4'

#control playback speed
frame_rate = 25

# vc = cv2.VideoCapture(0) #opens camera
vc = cv2.VideoCapture(NeedleViz_path2)

fgbg = cv2.createBackgroundSubtractorMOG2()

prev_frame_time = 0
new_frame_time = 0
average_fps_list = []

frameWidth = 440
frameHeight = 440
vc.set(3, frameWidth)
vc.set(4, frameHeight)

def empty(a):
  pass

if (vc.isOpened()== False): 
  print("Error opening video  file")

while(vc.isOpened()):
    rval, frame = vc.read()
    
    if rval == True:
        #Insert Code Here

        fgmask = fgbg.apply(frame)
     
        cv2.imshow('normal frame', frame)
        cv2.imshow('FGmask', fgmask)

        # Press Q on keyboard to  exit
        if cv2.waitKey(frame_rate) & 0xFF == ord('q'): #original waitkey is 25
            break
    
    #Break out of loop if video is done
    else:
        break  

vc.release() #Release the video capture object

# Close window
cv2.destroyWindow("normal frame")
cv2.destroyWindow("FGmask")